# <center><font color=#10e2e6>Kaggle Playground: Predicting Loan Approval</font></center>

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

train = pd.read_csv('/Users/poodle/Data Science Projects/Kaggle Playground/playground-series-s4e10/train.csv')
test = pd.read_csv('/Users/poodle/Data Science Projects/Kaggle Playground/playground-series-s4e10/test.csv')

In [26]:
train.dtypes

id                              int64
person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
loan_status                     int64
dtype: object

In [28]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
lbl.fit(train['person_home_ownership'])
train['person_home_ownership'] = lbl.transform(train['person_home_ownership'])
lbl.fit(train['loan_intent'])
train['loan_intent'] = lbl.transform(train['loan_intent'])
lbl.fit(train['loan_grade'])
train['loan_grade'] = lbl.transform(train['loan_grade'])
lbl.fit(train['cb_person_default_on_file'])
train['cb_person_default_on_file'] = lbl.transform(train['cb_person_default_on_file'])

lbl.fit(test['person_home_ownership'])
test['person_home_ownership'] = lbl.transform(test['person_home_ownership'])
lbl.fit(test['loan_intent'])
test['loan_intent'] = lbl.transform(test['loan_intent'])
lbl.fit(test['loan_grade'])
test['loan_grade'] = lbl.transform(test['loan_grade'])
lbl.fit(test['cb_person_default_on_file'])
test['cb_person_default_on_file'] = lbl.transform(test['cb_person_default_on_file'])

train

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,1,22,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,2,29,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,3,30,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,4,22,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,0,5.0,1,3,25000,15.95,0.21,1,10,0
58641,58641,28,28800,3,0.0,3,2,10000,12.73,0.35,0,8,1
58642,58642,23,44000,3,7.0,1,3,6800,16.00,0.15,0,2,1
58643,58643,22,30000,3,2.0,1,0,5000,8.90,0.17,0,3,0


In [29]:
from sklearn.model_selection import train_test_split

X = train.drop(['loan_status', 'id'], axis=1)
y = train['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=.3)

In [30]:
from sklearn import linear_model
from sklearn.metrics import roc_auc_score

lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)

predictions = model.predict(X_test)

print('ROC_AUC: ', roc_auc_score(y_test, predictions))

ROC_AUC:  0.8767290926721213


In [31]:
sublesson = pd.DataFrame()
sublesson['id'] = test.id

feats = test.select_dtypes(include=[np.number]).drop(['id'], axis=1).interpolate()
predictions = model.predict(feats)

sublesson['loan_status'] = predictions
sublesson.to_csv('PS_S4E10.csv', index=False)

sublesson

,id,loan_status
0,58645,0.908417
1,58646,0.092108
2,58647,0.482653
3,58648,0.046733
4,58649,0.227842
...,...,...
39093,97738,0.085224
39094,97739,0.047079
39095,97740,0.092368
39096,97741,0.624331
